# Zenital SQL and Python Assessment

## Data Gathering
Let's read the different Excel sheets and load them into data frames

In [18]:
import pandas as pd
import numpy as np

xlsx_file = pd.ExcelFile("dataset/technical-test-tables.xlsx")

### Messages

In [19]:
messages_df = pd.read_excel(
    xlsx_file, sheet_name="customer_courier_chat_messages "
)
messages_df.head()

,sender_app_type,customer_id,from_id,to_id,chat_started_by_message,order_id,order_stage,courier_id,message_sent_time
0,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:02:05
1,Courier IOS,99,21,99,False,555,ARRIVING,21,2022-08-09 08:01:15
2,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:00:10
3,Courier Android,122,87,122,True,38,ADDRESS_DELIVERY,87,2022-08-09 07:55:10
4,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:55:10


In [20]:
messages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sender_app_type          11 non-null     object        
 1   customer_id              11 non-null     int64         
 2   from_id                  11 non-null     int64         
 3   to_id                    11 non-null     int64         
 4   chat_started_by_message  11 non-null     bool          
 5   order_id                 11 non-null     int64         
 6   order_stage              11 non-null     object        
 7   courier_id               11 non-null     int64         
 8   message_sent_time        11 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](1), int64(5), object(2)
memory usage: 847.0+ bytes


In [21]:
orders_df = pd.read_excel(xlsx_file, sheet_name="orders")
orders_df

,order_id,city_code
0,38,BCN
1,134,OPO
2,555,BCN
3,875,VAL


In [22]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   order_id   4 non-null      int64 
 1   city_code  4 non-null      object
dtypes: int64(1), object(1)
memory usage: 196.0+ bytes


## Data Cleaning
The data frames are already cleaned and can be exported to the SQL data base as they are.

## Export to SQL DB

In [23]:
import sqlite3

cnx = sqlite3.connect("dataset/customer-courier.db")
messages_df.to_sql(
    name="customer_courier_chat_messages", con=cnx, if_exists="replace"
)
orders_df.to_sql(name="orders", con=cnx, if_exists="replace")

4

## Data Transformation
In this part I'll create a new data from the 2 gathered ones above frame containing the following fields
1. order_id
2. city_code
3. first_courier_message: Timestamp of the first courier message
4. first_customer_message: Timestamp of the first customer
message
5. num_messages_courier: Number of messages sent by courier
6. num_messages_customer: Number of messages sent by
customer
7. first_message_by: The first message sender (courier or customer)
8. conversation_started_at: Timestamp of the first message in the
conversation
9. first_responsetime_delay_seconds: Time (in secs) elapsed until
the first message was responded
10. last_message_time: Timestamp of the last message sent

11. last_message_order_stage: The stage of the order when the last
message was sent

In [24]:
merged_df = messages_df.merge(orders_df, how="left", on="order_id")
merged_df.head()

,sender_app_type,customer_id,from_id,to_id,chat_started_by_message,order_id,order_stage,courier_id,message_sent_time,city_code
0,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:02:05,BCN
1,Courier IOS,99,21,99,False,555,ARRIVING,21,2022-08-09 08:01:15,BCN
2,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:00:10,BCN
3,Courier Android,122,87,122,True,38,ADDRESS_DELIVERY,87,2022-08-09 07:55:10,BCN
4,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:55:10,VAL


In [25]:
merged_df["message_sent_by"] = np.where(
    merged_df["from_id"] == merged_df["courier_id"],
    "courier",
    "customer",
)

merged_df.head()

,sender_app_type,customer_id,from_id,to_id,chat_started_by_message,order_id,order_stage,courier_id,message_sent_time,city_code,message_sent_by
0,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:02:05,BCN,customer
1,Courier IOS,99,21,99,False,555,ARRIVING,21,2022-08-09 08:01:15,BCN,courier
2,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:00:10,BCN,customer
3,Courier Android,122,87,122,True,38,ADDRESS_DELIVERY,87,2022-08-09 07:55:10,BCN,courier
4,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:55:10,VAL,customer


In [28]:
grouped_df = merged_df.groupby("order_id")
grouped_df.head()

,sender_app_type,customer_id,from_id,to_id,chat_started_by_message,order_id,order_stage,courier_id,message_sent_time,city_code,message_sent_by
0,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:02:05,BCN,customer
1,Courier IOS,99,21,99,False,555,ARRIVING,21,2022-08-09 08:01:15,BCN,courier
2,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:00:10,BCN,customer
3,Courier Android,122,87,122,True,38,ADDRESS_DELIVERY,87,2022-08-09 07:55:10,BCN,courier
4,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:55:10,VAL,customer
5,Courier Android,43,75,43,False,875,ARRIVING,75,2022-08-07 14:53:10,VAL,courier
6,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:51:10,VAL,customer
7,Courier Android,43,75,43,True,875,ADDRESS_DELIVERY,75,2022-08-07 14:50:10,VAL,courier
8,Customer IOS,23,23,21,False,134,PICKING_UP,21,2022-08-07 10:02:05,OPO,customer
9,Courier IOS,23,21,23,False,134,ARRIVING,21,2022-08-07 10:01:15,OPO,courier


In [ ]:
grouped_df["first_courier_message"] = np.min()